### **SELENIUM CRAWLING** *(SmartStore)*

---

#### **SET UP**

##### *library*

In [18]:
import platform
import re, requests, csv, sys, os, zipfile

os_name = platform.system().lower()
architecture = platform.machine()

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
from time import sleep

In [19]:
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from dateutil import parser
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

##### *accept*

In [20]:
from glob import glob

driver_path = None
if os_name == 'darwin': # 맥 사용자
    driver_path = glob('../driver/**/chromedriver', recursive=True)[0]
else: # 윈도우 사용자
    driver_path = glob('../driver/**/chromedriver.exe', recursive=True)[0]

driver_path

'../driver\\chromedriver-win64\\chromedriver.exe'

In [21]:
# 드라이버 파일의 권한을 확인하고 수정
if os_name == 'darwin':  # 맥 사용자
    driver_permissions = os.stat(driver_path).st_mode
    if not (driver_permissions & 0o100):  # 실행 권한 확인
        os.chmod(driver_path, driver_permissions | 0o111)  # 실행 권한 추가
else:  # 윈도우 사용자
    driver_permissions = os.stat(driver_path).st_mode
    if not (driver_permissions & 0o100):  # 실행 권한 확인
        os.chmod(driver_path, driver_permissions | 0o111)  # 실행 권한 추가


In [22]:
service = Service(executable_path=driver_path)
chrome_options =  webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')

#### **WORK SPACE**

In [23]:
url = 'https://smartstore.naver.com/rkrguswoo/products/10244529817'

##### *chrome setting*

In [24]:
# Web OPEN
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url)

In [25]:
# Web ZOOM
driver.execute_script("document.body.style.zoom='10%'")
# Web Height
last_height = driver.execute_script("return document.body.scrollHeight")

In [26]:
# Page Loading

while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(1)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

##### *Crawling*

In [27]:
# Parsing
soup = BeautifulSoup(driver.page_source, 'html.parser')
data = []

In [28]:
# Count page
count_page = (soup.select_one('#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a > span').text)
page_int = int(count_page.replace(',', ''))
page_no = (page_int + 20 - 1)//20

In [29]:
# Page Selector
selector_page = soup.select_one('#REVIEW  a[data-shp-contents-id=\"1\"]')

In [30]:
# Title selector
selector_title = soup.select_one('title')

# Title Text
selector_title.text

'홀베리 1일1레몬 100% 유기농 레몬즙 착즙액 14포 1박스 : 화수분'

In [31]:
# Review selector
good = ('#REVIEW div._1McWUwk15j')
selector = soup.select_one(good)

In [32]:
# Id selector
id = selector.select_one('strong._2L3vDiadT9').text

In [33]:
# Created selector
created = parser.parse(selector.select_one('span._2L3vDiadT9').text)

In [34]:
created

datetime.datetime(2003, 10, 24, 0, 0)

In [35]:
# Rate selector
rate = selector.select_one('em').text

In [36]:
# content selector
content = selector.select_one('._3z6gI4oI6l').text

In [37]:
#교수님이 작성해주신 코드 (담주에 다시 물어보기)

#result_df = pd.DataFrame([{
#    'ID' : id,
#    'RATE' : rate,
#    'CREATED': created,
#    'REVIEW': content
#}
#    for selector in soup.select(good)
#])

In [38]:
for page in range(1, page_no):
    # 페이지 로드 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#REVIEW'))
    )

    time.sleep(3)

    # 현재 페이지의 HTML 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 리뷰 데이터 수집
    good = '#REVIEW div._1McWUwk15j'
    for selector in soup.select(good):
        id_selector = selector.select_one('strong._2L3vDiadT9').text
        created_selector = parser.parse(selector.select_one('span._2L3vDiadT9').text)
        rate_selector = selector.select_one('em').text
        content_selector = selector.select_one('._3z6gI4oI6l').text

        # 각 리뷰 정보를 딕셔너리로 저장
        data.append({
            'ID': id_selector,
            'RATE': rate_selector,
            'CREATED': created_selector,
            'REVIEW': content_selector
        })

    # 다음 페이지 버튼 클릭
    if page < (page_no+1):  # 마지막 페이지에서는 다음 페이지 버튼 클릭을 시도하지 않음
        next_page_element = driver.find_element(By.CSS_SELECTOR, f'#REVIEW  a[data-shp-contents-id="{page + 1}"]')
        driver.execute_script("arguments[0].click();", next_page_element)

In [39]:
result_df = pd.DataFrame(data)
result_df['CREATED'] = result_df['CREATED'].apply(lambda x: f"{x.day:02d}-{x.month:02d}-{str(x.year)[2:]}")

In [40]:
driver.close()
driver.quit()

In [46]:
file_name = selector_title.text[:3]

In [48]:
result_df.to_csv(rf'..\data\{file_name}_reviews.csv', index=False, encoding='utf-8-sig')